In [1]:
# import dependencies
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
# Load the popularity and biodiversity data
file_path = Path('Resources/parks_pop_bio.csv')
pop_bio_df = pd.read_csv(file_path)
pop_bio_df.head()

,park_code,park_name,state,acres,latitude,longitude,trail_count,avg_popularity,species_count
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,179,8.376,1709
1,ARCH,Arches National Park,UT,76519,38.68,-109.57,48,13.462,1048
2,BADL,Badlands National Park,SD,242756,43.75,-102.50,10,13.141,1389
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25,70,7.176,2269
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08,4,4.055,1726


In [3]:
# Load the trail csv
file_path = Path('Resources/parks_trails_final.csv')
trail_df = pd.read_csv(file_path)
trail_df.head()

,park_code,park_name,state,acres,latitude,longitude,trail_count,avg_popularity
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21,179.0,8.375920
1,ARCH,Arches National Park,UT,76519,38.68,-109.57,48.0,13.462423
2,BADL,Badlands National Park,SD,242756,43.75,-102.50,10.0,13.141040
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25,70.0,7.176380
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08,4.0,4.055200


In [4]:
# Determine the shape of the data
print(pop_bio_df.shape)
print(trail_df.shape)

(52, 9)
(52, 8)


In [9]:
# Determine if they're null rows
pop_bio_df.info()
print("-------------------------------------------------------")
trail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   park_code       52 non-null     object 
 1   park_name       52 non-null     object 
 2   state           52 non-null     object 
 3   acres           52 non-null     int64  
 4   latitude        52 non-null     float64
 5   longitude       52 non-null     float64
 6   trail_count     52 non-null     int64  
 7   avg_popularity  52 non-null     float64
 8   species_count   52 non-null     int64  
dtypes: float64(3), int64(3), object(3)
memory usage: 3.8+ KB
-------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   park_code       52 non-null     object 
 1   park_name       52 non-null     object 
 2 